In [1]:
# this file is used to search for the optimal parameters 
# among all the combination of three kernels and c = [0.01,0.1,1]
# we first determine the best kernel, and then search for the optimal value of c
import numpy as np
from numpy import linalg
from dataloader import *
from svmmodel import *
from datetime import datetime
from sklearn.model_selection import ShuffleSplit

In [2]:
# load data
train_x, train_y, test_x, test_y = loadcatech()
xtrain=xtrain.astype(np.double)
ytrain=ytrain.astype(np.double)

# since caltech dataset is too small, we use all of them to do the cross validation
X = np.concatenate((train_x,test_x), axis=0)
ytrain = np.concatenate((train_y,test_y), axis=0)

#dimension reduction by PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=35)
pca.fit(X)
xtrain = pca.transform(X)

#normalization--change all the negative value to positive
#mean = xtrain.mean()
#var  = xtrain.var()
#xtrain= (xtrain - mean)/var
#minvalue = xtrain.min()
#xtrain = xtrain -minvalue
print(xtrain.shape)
print(ytrain.shape)
#exitFlag = 0

(646, 35)
(646,)


In [3]:
# main function for svm caltech
# sorry for not changing the function name, it should be runsvmcaltech
# but the code is right, i build this code just as the function is Runnsvmnist, you can just run it
# the variable "result" is the value of time running this svm function
def Runsvmmnist(k,const,trainxdata,trainydata):
    a = datetime.now()
    name=['linear','gaussian','poly_2']
    if k ==0:
        kern = linear_kernel
    if k ==1:
        kern = gaussian_kernel
    if k ==2:
        kern = polynomial_kernel
    rs = ShuffleSplit(n_splits=5, test_size=.2, random_state=0)
    accuracy = 0
    for train_index, test_index in rs.split(trainxdata):
        train_x=trainxdata[train_index]
        train_y=trainydata[train_index]
        test_x=trainxdata[test_index]
        test_y=trainydata[test_index]
# create, train and do prediction with multiple svms for each categories
        final_result = np.zeros(test_y.shape[0])
        correct=0
        list = [SVM(C=const,kernel=kern) for i in range(0,10)]
        for i in range(0,10):
            #split dataset into 10 small folders for each category
            #traindata  = train_x[k:(i+1)*6000,:]
            #trainlabel = train_y[k:(i+1)*6000] - i + 1
            traindata = train_x
            trainlabel = train_y -i +1

        #for catech dataset, don't need to split the dataset, just use the whole traindata 10 times, so the code should be
        #traindata = train_x
        #trainlabel = train_y - i + 1

            trainlabel=np.array(np.where(trainlabel == 1, trainlabel, -1))
            #k=k+6000
            list[i].fit(traindata, trainlabel)
            predictions = list[i].predict(test_x)
            final_result=np.array(np.where(predictions !=1, final_result, i))
        correct = np.sum(final_result == test_y)
        correct = float(correct)/(test_y.shape[0])
        accuracy = accuracy + correct
    accuracy = float(accuracy) / 5
    b = datetime.now()
#calculate the accuracy of our prediction
    result = (b-a).seconds
    print("k=",name[k],"c=",const,accuracy,result)
    return accuracy, result

In [4]:
# two for loops is appied for the different combination of two parameters
c=np.array([0.01,0.1,1])
accuracy = np.zeros(12)
result = np.zeros(12)
count=0
name=['linear','gaussian','poly_2']
for k in range(0,3):
    for i in range(0,3):
        accuracy[count], result[count]= Runsvmmnist(k,c[i],xtrain,ytrain)
        count = count +1

     pcost       dcost       gap    pres   dres
 0: -6.5867e+01 -1.0562e+01  4e+03  6e+01  4e-13
 1: -1.8409e+00 -1.0475e+01  6e+01  9e-01  3e-13
 2: -1.0082e+00 -7.1027e+00  1e+01  1e-01  4e-14
 3: -8.0281e-01 -3.6156e+00  5e+00  4e-02  1e-14
 4: -6.8038e-01 -1.6990e+00  1e+00  1e-02  5e-15
 5: -6.5845e-01 -1.1689e+00  7e-01  4e-03  3e-15
 6: -6.8904e-01 -8.3763e-01  2e-01  9e-04  3e-15
 7: -7.0708e-01 -7.6232e-01  6e-02  1e-04  3e-15
 8: -7.1994e-01 -7.3659e-01  2e-02  3e-05  3e-15
 9: -7.2426e-01 -7.2870e-01  5e-03  4e-06  3e-15
10: -7.2590e-01 -7.2625e-01  4e-04  3e-07  4e-15
11: -7.2604e-01 -7.2606e-01  2e-05  1e-08  4e-15
12: -7.2604e-01 -7.2604e-01  3e-07  2e-10  4e-15
Optimal solution found.
100 support vectors out of 516 points
     pcost       dcost       gap    pres   dres
 0: -6.8107e+01 -1.0755e+01  4e+03  6e+01  4e-13
 1: -1.8175e+00 -1.0669e+01  6e+01  8e-01  4e-13
 2: -1.0393e+00 -6.9556e+00  1e+01  1e-01  5e-14
 3: -8.4068e-01 -2.9203e+00  3e+00  2e-02  1e-14
 4: -8.06

12: -8.2174e-01 -8.2174e-01  1e-08  8e-12  4e-15
Optimal solution found.
103 support vectors out of 516 points
     pcost       dcost       gap    pres   dres
 0: -6.4469e+01 -1.0814e+01  4e+03  6e+01  3e-13
 1: -1.7470e+00 -1.0727e+01  6e+01  8e-01  4e-13
 2: -9.8007e-01 -7.0699e+00  1e+01  1e-01  5e-14
 3: -7.7114e-01 -2.7790e+00  3e+00  2e-02  1e-14
 4: -7.2820e-01 -1.4732e+00  1e+00  7e-03  4e-15
 5: -7.5164e-01 -9.7011e-01  3e-01  1e-03  4e-15
 6: -7.8278e-01 -8.5440e-01  8e-02  3e-04  3e-15
 7: -7.9436e-01 -8.1801e-01  2e-02  1e-05  4e-15
 8: -8.0218e-01 -8.0689e-01  5e-03  2e-06  4e-15
 9: -8.0394e-01 -8.0440e-01  5e-04  7e-08  4e-15
10: -8.0413e-01 -8.0416e-01  2e-05  3e-09  4e-15
11: -8.0414e-01 -8.0414e-01  4e-07  6e-11  4e-15
Optimal solution found.
104 support vectors out of 516 points
     pcost       dcost       gap    pres   dres
 0: -6.8070e+01 -1.1046e+01  4e+03  6e+01  3e-13
 1: -1.7409e+00 -1.0959e+01  6e+01  8e-01  3e-13
 2: -9.6670e-01 -6.9732e+00  1e+01  9e-02  4e

     pcost       dcost       gap    pres   dres
 0: -6.2803e+01 -1.1173e+01  4e+03  7e+01  3e-13
 1: -1.6148e+00 -1.1084e+01  6e+01  8e-01  3e-13
 2: -8.9172e-01 -7.0534e+00  1e+01  9e-02  4e-14
 3: -7.2173e-01 -3.0540e+00  3e+00  2e-02  1e-14
 4: -6.7885e-01 -1.3570e+00  8e-01  5e-03  3e-15
 5: -7.2385e-01 -9.3893e-01  2e-01  8e-04  3e-15
 6: -7.4956e-01 -8.4864e-01  1e-01  3e-04  3e-15
 7: -7.6610e-01 -8.0757e-01  4e-02  5e-05  3e-15
 8: -7.7565e-01 -7.8896e-01  1e-02  9e-06  3e-15
 9: -7.7972e-01 -7.8236e-01  3e-03  2e-06  3e-15
10: -7.8063e-01 -7.8094e-01  3e-04  2e-07  3e-15
11: -7.8075e-01 -7.8076e-01  1e-05  6e-09  3e-15
12: -7.8075e-01 -7.8075e-01  2e-07  1e-10  4e-15
Optimal solution found.
99 support vectors out of 516 points
     pcost       dcost       gap    pres   dres
 0: -7.1295e+01 -1.0819e+01  4e+03  6e+01  4e-13
 1: -1.8813e+00 -1.0732e+01  6e+01  8e-01  4e-13
 2: -1.0728e+00 -6.9882e+00  1e+01  1e-01  5e-14
 3: -8.8605e-01 -3.6494e+00  4e+00  4e-02  2e-14
 4: -7.597

     pcost       dcost       gap    pres   dres
 0: -6.9312e+01 -1.0572e+01  4e+03  6e+01  4e-13
 1: -1.8444e+00 -1.0485e+01  6e+01  8e-01  4e-13
 2: -1.0525e+00 -6.8232e+00  1e+01  1e-01  5e-14
 3: -8.4447e-01 -3.1493e+00  4e+00  3e-02  1e-14
 4: -7.5980e-01 -1.8493e+00  2e+00  1e-02  6e-15
 5: -7.6466e-01 -1.0757e+00  4e-01  3e-03  3e-15
 6: -7.7995e-01 -9.0097e-01  1e-01  5e-04  4e-15
 7: -8.0008e-01 -8.4098e-01  4e-02  1e-04  4e-15
 8: -8.0407e-01 -8.2349e-01  2e-02  8e-17  4e-15
 9: -8.0994e-01 -8.1617e-01  6e-03  5e-17  3e-15
10: -8.1231e-01 -8.1324e-01  9e-04  4e-17  4e-15
11: -8.1272e-01 -8.1276e-01  4e-05  4e-17  4e-15
12: -8.1274e-01 -8.1274e-01  5e-07  5e-17  4e-15
Optimal solution found.
103 support vectors out of 516 points
     pcost       dcost       gap    pres   dres
 0: -7.3157e+01 -1.1385e+01  4e+03  7e+01  4e-13
 1: -1.9390e+00 -1.1298e+01  7e+01  9e-01  4e-13
 2: -1.0716e+00 -7.6705e+00  1e+01  1e-01  5e-14
 3: -8.4938e-01 -2.8367e+00  3e+00  2e-02  8e-15
 4: -8.37

 9: -8.8614e-01 -8.8846e-01  2e-03  8e-07  4e-15
10: -8.8703e-01 -8.8730e-01  3e-04  8e-08  4e-15
11: -8.8714e-01 -8.8716e-01  2e-05  4e-09  5e-15
12: -8.8715e-01 -8.8715e-01  5e-07  8e-11  5e-15
Optimal solution found.
115 support vectors out of 516 points
     pcost       dcost       gap    pres   dres
 0: -6.7176e+01 -1.0794e+01  4e+03  6e+01  3e-13
 1: -1.7201e+00 -1.0706e+01  6e+01  8e-01  3e-13
 2: -9.5412e-01 -6.7540e+00  1e+01  9e-02  4e-14
 3: -7.7486e-01 -2.6297e+00  3e+00  2e-02  8e-15
 4: -7.6332e-01 -1.2162e+00  5e-01  2e-03  4e-15
 5: -8.0518e-01 -1.0481e+00  3e-01  1e-03  3e-15
 6: -8.1884e-01 -9.4781e-01  1e-01  1e-05  4e-15
 7: -8.4116e-01 -9.0478e-01  6e-02  4e-17  4e-15
 8: -8.5523e-01 -8.7854e-01  2e-02  4e-17  4e-15
 9: -8.6117e-01 -8.6871e-01  8e-03  4e-17  4e-15
10: -8.6335e-01 -8.6565e-01  2e-03  9e-17  4e-15
11: -8.6414e-01 -8.6459e-01  4e-04  5e-17  4e-15
12: -8.6432e-01 -8.6435e-01  3e-05  6e-17  4e-15
13: -8.6434e-01 -8.6434e-01  5e-07  7e-17  4e-15
Optimal 

...

     pcost       dcost       gap    pres   dres
 0: -7.3011e-05 -5.1667e+02  2e+03  1e+00  8e-14
 1:  1.8423e-02 -1.3489e+02  1e+02  2e-02  8e-14
 2:  9.8478e-03 -1.1963e+01  1e+01  1e-03  9e-14
 3:  1.4707e-02 -4.2811e+00  5e+00  4e-04  1e-13
 4:  4.1453e-03 -7.0041e-01  8e-01  6e-05  8e-14
 5:  3.8834e-03 -2.6690e-01  3e-01  2e-05  4e-14
 6:  1.2904e-03 -6.9309e-02  8e-02  4e-06  1e-14
 7: -7.5191e-04 -1.0318e-02  1e-02  3e-07  5e-15
 8: -1.5408e-03 -4.0833e-03  3e-03  8e-08  3e-15
 9: -1.8450e-03 -2.3274e-03  5e-04  4e-09  3e-15
10: -1.9456e-03 -2.0331e-03  9e-05  9e-11  3e-15
11: -1.9705e-03 -1.9758e-03  5e-06  2e-12  3e-15
12: -1.9723e-03 -1.9725e-03  1e-07  4e-14  4e-15
13: -1.9724e-03 -1.9724e-03  5e-09  9e-16  4e-15
Optimal solution found.
80 support vectors out of 516 points
     pcost       dcost       gap    pres   dres
 0: -2.9991e-05 -5.1665e+02  2e+03  1e+00  6e-14
 1:  8.9488e-03 -1.3488e+02  1e+02  2e-02  8e-14
 2:  5.5573e-03 -1.3134e+01  1e+01  2e-03  8e-14
 3:  1.244

     pcost       dcost       gap    pres   dres
 0: -2.6544e-05 -5.1648e+02  2e+03  1e+00  5e-14
 1:  8.8361e-03 -1.3461e+02  1e+02  2e-02  6e-14
 2:  5.9182e-03 -1.0744e+01  1e+01  1e-03  8e-14
 3:  1.7127e-03 -2.0071e+00  2e+00  2e-04  5e-14
 4:  5.9783e-04 -3.4391e-01  4e-01  3e-05  3e-14
 5:  3.6058e-04 -9.2133e-02  1e-01  8e-06  1e-14
 6: -2.2040e-04 -2.0527e-02  2e-02  1e-06  4e-15
 7: -7.4230e-04 -3.6730e-03  3e-03  2e-07  2e-15
 8: -9.3431e-04 -1.3632e-03  4e-04  2e-16  2e-15
 9: -1.0155e-03 -1.1118e-03  1e-04  2e-16  2e-15
10: -1.0391e-03 -1.0542e-03  2e-05  2e-16  2e-15
11: -1.0438e-03 -1.0446e-03  8e-07  2e-16  2e-15
12: -1.0441e-03 -1.0441e-03  2e-08  2e-16  2e-15
Optimal solution found.
53 support vectors out of 516 points
     pcost       dcost       gap    pres   dres
 0: -7.0631e-06 -5.1625e+02  2e+03  1e+00  8e-14
 1:  3.5789e-03 -1.3426e+02  1e+02  2e-02  1e-13
 2:  2.8744e-03 -7.2239e+00  8e+00  8e-04  1e-13
 3:  4.6312e-04 -1.0337e+00  1e+00  1e-04  7e-14
 4:  1.817

     pcost       dcost       gap    pres   dres
 0: -9.4337e-06 -5.1631e+02  2e+03  1e+00  6e-14
 1:  4.0515e-03 -1.3435e+02  1e+02  2e-02  7e-14
 2:  3.0485e-03 -7.7870e+00  8e+00  9e-04  1e-13
 3:  1.5394e-04 -1.2330e+00  1e+00  1e-04  7e-14
 4:  1.9316e-03 -5.0600e-01  6e-01  5e-05  2e-14
 5:  8.7532e-04 -7.5793e-02  9e-02  6e-06  9e-15
 6:  2.0777e-04 -1.3370e-02  1e-02  1e-06  4e-15
 7: -1.9872e-04 -1.3664e-03  1e-03  5e-08  2e-15
 8: -3.0644e-04 -6.3290e-04  3e-04  2e-16  1e-15
 9: -3.5772e-04 -4.5347e-04  1e-04  2e-16  1e-15
10: -3.7693e-04 -4.0056e-04  2e-05  2e-16  1e-15
11: -3.8291e-04 -3.8724e-04  4e-06  2e-16  1e-15
12: -3.8421e-04 -3.8462e-04  4e-07  2e-16  1e-15
13: -3.8436e-04 -3.8436e-04  7e-09  2e-16  1e-15
Optimal solution found.
21 support vectors out of 516 points
     pcost       dcost       gap    pres   dres
 0: -6.0860e-05 -5.1665e+02  2e+03  1e+00  9e-14
 1:  1.5853e-02 -1.3486e+02  1e+02  2e-02  9e-14
 2:  9.3759e-03 -1.4409e+01  2e+01  2e-03  9e-14
 3:  7.698

     pcost       dcost       gap    pres   dres
 0: -5.2957e-05 -5.1665e+02  2e+03  1e+00  2e-13
 1:  1.4969e-02 -1.3487e+02  1e+02  2e-02  2e-13
 2:  9.4487e-03 -1.3423e+01  1e+01  2e-03  3e-13
 3:  3.6278e-03 -3.1540e+00  3e+00  3e-04  2e-13
 4:  5.7683e-04 -6.9138e-01  8e-01  7e-05  7e-14
 5: -2.4968e-05 -1.5179e-01  2e-01  1e-05  2e-14
 6: -1.3248e-04 -4.0293e-02  4e-02  3e-06  1e-14
 7: -9.0699e-04 -9.3467e-03  9e-03  4e-07  5e-15
 8: -1.3580e-03 -3.1075e-03  2e-03  7e-08  3e-15
 9: -1.5356e-03 -1.8803e-03  3e-04  9e-09  4e-15
10: -1.5978e-03 -1.6460e-03  5e-05  4e-10  4e-15
11: -1.6111e-03 -1.6149e-03  4e-06  2e-11  5e-15
12: -1.6124e-03 -1.6125e-03  8e-08  3e-13  5e-15
Optimal solution found.
77 support vectors out of 516 points
     pcost       dcost       gap    pres   dres
 0: -9.2623e-05 -5.1726e+02  2e+03  1e+00  1e-13
 1:  1.9411e-02 -1.3579e+02  2e+02  2e-02  1e-13
 2:  1.0996e-02 -1.6011e+01  2e+01  2e-03  1e-13
 3:  7.7453e-03 -3.2278e+00  4e+00  3e-04  1e-13
 4:  5.743

     pcost       dcost       gap    pres   dres
 0: -7.6332e-05 -5.1697e+02  2e+03  1e+00  9e-14
 1:  1.9041e-02 -1.3534e+02  1e+02  2e-02  1e-13
 2:  1.1815e-02 -1.4500e+01  2e+01  2e-03  1e-13
 3:  3.7257e-03 -3.2220e+00  4e+00  4e-04  8e-14
 4:  3.8863e-03 -8.5770e-01  1e+00  8e-05  4e-14
 5:  2.3854e-03 -2.2377e-01  3e-01  2e-05  2e-14
 6:  1.3650e-03 -5.6181e-02  6e-02  3e-06  1e-14
 7: -1.1353e-03 -3.4611e-03  2e-03  3e-08  6e-15
 8: -1.6214e-03 -2.5421e-03  9e-04  9e-09  3e-15
 9: -1.8752e-03 -2.1236e-03  2e-04  1e-09  4e-15
10: -1.9529e-03 -1.9928e-03  4e-05  1e-10  4e-15
11: -1.9660e-03 -1.9728e-03  7e-06  2e-11  4e-15
12: -1.9686e-03 -1.9691e-03  5e-07  2e-16  4e-15
13: -1.9688e-03 -1.9688e-03  9e-09  2e-16  4e-15
Optimal solution found.
86 support vectors out of 516 points
     pcost       dcost       gap    pres   dres
 0: -3.5393e-06 -5.1625e+02  2e+03  1e+00  6e-14
 1:  1.4727e-03 -1.3426e+02  1e+02  2e-02  1e-13
 2:  1.1625e-03 -4.9959e+00  5e+00  6e-04  9e-14
 3:  2.164

In [5]:
# show the result
count = 0
for k in range(0,3):
    for i in range(0,3):
        print("k=",name[k],"c=",c[i],accuracy[count],result[count])
        count = count +1

k= linear c= 0.01 0.6 32.0
k= linear c= 0.1 0.583076923077 31.0
k= linear c= 1.0 0.572307692308 32.0
k= gaussian c= 0.01 0.0969230769231 149.0
k= gaussian c= 0.1 0.0969230769231 133.0
k= gaussian c= 1.0 0.101538461538 142.0
k= poly_2 c= 0.01 0.352307692308 39.0
k= poly_2 c= 0.1 0.352307692308 41.0
k= poly_2 c= 1.0 0.352307692308 40.0


In [11]:
# after we find the optimal parameters which is linear_kernel and c=0.01
# we do cross validation again to find the best c when kernel is linear kernel
c1=np.array([0.005,0.01,0.03,0.05,0.07,0.09])
accuracy1 = np.zeros(6)
result1 = np.zeros(6)
count = 0
for i in range(0,6):        
    accuracy1[count], result1[count]= Runsvmmnist(0,c1[i],xtrain,ytrain)
    count = count +1

     pcost       dcost       gap    pres   dres
 0: -6.5541e+01 -5.2822e+00  4e+03  6e+01  4e-13
 1: -1.2445e+00 -5.2603e+00  5e+01  7e-01  3e-13
 2: -5.3981e-01 -4.0326e+00  8e+00  8e-02  4e-14
 3: -4.1353e-01 -1.9380e+00  2e+00  2e-02  1e-14
 4: -3.5459e-01 -8.6166e-01  7e-01  6e-03  3e-15
 5: -3.6328e-01 -5.6548e-01  2e-01  2e-03  2e-15
 6: -3.8123e-01 -4.6455e-01  1e-01  6e-04  2e-15
 7: -3.9333e-01 -4.1907e-01  3e-02  1e-04  2e-15
 8: -3.9869e-01 -4.0585e-01  8e-03  2e-05  2e-15
 9: -4.0087e-01 -4.0201e-01  1e-03  3e-06  2e-15
10: -4.0128e-01 -4.0133e-01  5e-05  1e-07  2e-15
11: -4.0130e-01 -4.0130e-01  9e-07  2e-09  2e-15
12: -4.0130e-01 -4.0130e-01  9e-09  2e-11  2e-15
Optimal solution found.
107 support vectors out of 516 points
     pcost       dcost       gap    pres   dres
 0: -6.7770e+01 -5.3804e+00  4e+03  6e+01  4e-13
 1: -1.2457e+00 -5.3588e+00  5e+01  7e-01  4e-13
 2: -5.4546e-01 -4.0379e+00  7e+00  6e-02  4e-14
 3: -4.2586e-01 -1.6205e+00  2e+00  1e-02  8e-15
 4: -4.20

     pcost       dcost       gap    pres   dres
 0: -6.4149e+01 -5.4099e+00  4e+03  6e+01  4e-13
 1: -1.1958e+00 -5.3881e+00  5e+01  7e-01  3e-13
 2: -5.2332e-01 -4.0979e+00  8e+00  7e-02  4e-14
 3: -3.9582e-01 -1.6472e+00  2e+00  1e-02  7e-15
 4: -3.8099e-01 -7.3088e-01  4e-01  3e-03  2e-15
 5: -3.9929e-01 -5.2528e-01  1e-01  5e-04  2e-15
 6: -4.1998e-01 -4.6545e-01  5e-02  1e-04  2e-15
 7: -4.2990e-01 -4.4251e-01  1e-02  2e-05  2e-15
 8: -4.3339e-01 -4.3635e-01  3e-03  4e-06  2e-15
 9: -4.3445e-01 -4.3466e-01  2e-04  2e-08  2e-15
10: -4.3454e-01 -4.3455e-01  7e-06  7e-10  2e-15
11: -4.3454e-01 -4.3454e-01  1e-07  9e-12  2e-15
Optimal solution found.
111 support vectors out of 516 points
     pcost       dcost       gap    pres   dres
 0: -6.7733e+01 -5.5231e+00  4e+03  6e+01  3e-13
 1: -1.2086e+00 -5.5013e+00  5e+01  7e-01  3e-13
 2: -5.0269e-01 -4.1068e+00  6e+00  5e-02  2e-14
 3: -4.0304e-01 -1.6801e+00  2e+00  1e-02  5e-15
 4: -3.9775e-01 -7.7494e-01  4e-01  3e-03  2e-15
 5: -4.22

13: -4.0685e-01 -4.0685e-01  3e-07  5e-17  2e-15
Optimal solution found.
105 support vectors out of 516 points
     pcost       dcost       gap    pres   dres
 0: -7.0942e+01 -5.4102e+00  4e+03  6e+01  4e-13
 1: -1.2937e+00 -5.3886e+00  5e+01  7e-01  4e-13
 2: -5.6249e-01 -4.0606e+00  7e+00  6e-02  4e-14
 3: -4.4629e-01 -1.9764e+00  2e+00  2e-02  1e-14
 4: -3.8701e-01 -1.0224e+00  9e-01  6e-03  4e-15
 5: -3.9571e-01 -5.9991e-01  2e-01  1e-03  2e-15
 6: -4.1728e-01 -4.9511e-01  9e-02  4e-04  2e-15
 7: -4.2670e-01 -4.6193e-01  4e-02  1e-04  2e-15
 8: -4.3368e-01 -4.4438e-01  1e-02  1e-05  2e-15
 9: -4.3611e-01 -4.4023e-01  4e-03  3e-06  2e-15
10: -4.3736e-01 -4.3841e-01  1e-03  5e-07  2e-15
11: -4.3779e-01 -4.3783e-01  4e-05  9e-09  3e-15
12: -4.3781e-01 -4.3781e-01  9e-07  1e-10  3e-15
13: -4.3781e-01 -4.3781e-01  2e-08  3e-12  3e-15
Optimal solution found.
117 support vectors out of 516 points
     pcost       dcost       gap    pres   dres
 0: -6.8566e+01 -5.4411e+00  4e+03  6e+01  4e

     pcost       dcost       gap    pres   dres
 0: -7.2795e+01 -5.6909e+00  4e+03  7e+01  4e-13
 1: -1.3303e+00 -5.6692e+00  5e+01  8e-01  4e-13
 2: -5.6702e-01 -4.3726e+00  8e+00  8e-02  4e-14
 3: -4.2517e-01 -1.6393e+00  2e+00  9e-03  5e-15
 4: -4.2568e-01 -7.0385e-01  3e-01  2e-03  2e-15
 5: -4.4339e-01 -5.9282e-01  2e-01  5e-04  2e-15
 6: -4.6038e-01 -5.3852e-01  8e-02  2e-04  2e-15
 7: -4.7338e-01 -4.9719e-01  2e-02  3e-05  2e-15
 8: -4.7907e-01 -4.8651e-01  8e-03  4e-06  2e-15
 9: -4.8119e-01 -4.8319e-01  2e-03  1e-06  2e-15
10: -4.8192e-01 -4.8205e-01  1e-04  1e-08  2e-15
11: -4.8198e-01 -4.8198e-01  3e-06  2e-10  2e-15
12: -4.8198e-01 -4.8198e-01  9e-08  5e-12  2e-15
Optimal solution found.
115 support vectors out of 516 points
     pcost       dcost       gap    pres   dres
 0: -3.5347e+01 -5.1485e+00  4e+03  6e+01  3e-13
 1: -6.5512e-01 -5.1251e+00  5e+01  7e-01  3e-13
 2: -2.9083e-01 -3.7108e+00  7e+00  6e-02  3e-14
 3: -1.9146e-01 -1.3335e+00  2e+00  1e-02  5e-15
 4: -1.59

     pcost       dcost       gap    pres   dres
 0: -4.0889e+01 -5.0367e+00  4e+03  6e+01  4e-13
 1: -7.5970e-01 -5.0136e+00  4e+01  7e-01  3e-13
 2: -3.3687e-01 -3.6378e+00  7e+00  6e-02  3e-14
 3: -2.2785e-01 -1.3094e+00  2e+00  1e-02  6e-15
 4: -2.0012e-01 -4.8098e-01  4e-01  2e-03  2e-15
 5: -2.1360e-01 -2.9220e-01  9e-02  4e-04  2e-15
 6: -2.2610e-01 -2.5369e-01  3e-02  9e-05  2e-15
 7: -2.3239e-01 -2.3914e-01  7e-03  9e-06  2e-15
 8: -2.3446e-01 -2.3564e-01  1e-03  9e-07  2e-15
 9: -2.3494e-01 -2.3498e-01  4e-05  6e-09  2e-15
10: -2.3496e-01 -2.3496e-01  9e-07  1e-10  2e-15
11: -2.3496e-01 -2.3496e-01  3e-08  3e-12  2e-15
Optimal solution found.
69 support vectors out of 516 points
     pcost       dcost       gap    pres   dres
 0: -5.3256e+01 -5.4637e+00  4e+03  6e+01  3e-13
 1: -9.4883e-01 -5.4410e+00  5e+01  7e-01  3e-13
 2: -3.9227e-01 -4.0101e+00  7e+00  5e-02  3e-14
 3: -3.1049e-01 -1.2585e+00  1e+00  7e-03  4e-15
 4: -3.1178e-01 -6.2287e-01  4e-01  2e-03  2e-15
 5: -3.250

...

13: -6.6762e+00 -6.6763e+00  6e-05  3e-09  3e-14
14: -6.6762e+00 -6.6762e+00  7e-07  3e-11  3e-14
Optimal solution found.
95 support vectors out of 516 points
     pcost       dcost       gap    pres   dres
 0: -7.5567e+01 -9.8248e+01  4e+03  3e+01  4e-13
 1: -1.0943e+01 -9.1699e+01  2e+02  1e+00  3e-13
 2: -8.1393e+00 -4.3166e+01  6e+01  2e-01  7e-14
 3: -6.9913e+00 -2.4676e+01  3e+01  9e-02  4e-14
 4: -6.3590e+00 -1.5064e+01  1e+01  4e-02  3e-14
 5: -6.0345e+00 -1.0155e+01  6e+00  1e-02  3e-14
 6: -6.0341e+00 -7.4212e+00  2e+00  4e-03  3e-14
 7: -6.0302e+00 -6.6904e+00  8e-01  8e-04  3e-14
 8: -6.1533e+00 -6.3670e+00  2e-01  2e-04  3e-14
 9: -6.1850e+00 -6.2618e+00  8e-02  2e-05  4e-14
10: -6.2117e+00 -6.2235e+00  1e-02  7e-07  4e-14
11: -6.2165e+00 -6.2177e+00  1e-03  6e-08  4e-14
12: -6.2170e+00 -6.2171e+00  4e-05  1e-09  4e-14
13: -6.2171e+00 -6.2171e+00  4e-07  2e-11  4e-14
Optimal solution found.
90 support vectors out of 516 points
     pcost       dcost       gap    pres   dre

13: -7.4376e+00 -7.4376e+00  7e-07  4e-16  3e-14
Optimal solution found.
106 support vectors out of 516 points
     pcost       dcost       gap    pres   dres
 0: -8.0698e+01 -1.0104e+02  4e+03  3e+01  4e-13
 1: -1.1179e+01 -9.4506e+01  2e+02  1e+00  4e-13
 2: -8.3551e+00 -4.2456e+01  5e+01  2e-01  8e-14
 3: -7.7232e+00 -2.0672e+01  2e+01  5e-02  4e-14
 4: -7.7406e+00 -1.1402e+01  4e+00  1e-02  3e-14
 5: -8.0852e+00 -9.6635e+00  2e+00  3e-03  3e-14
 6: -8.3248e+00 -8.9671e+00  7e-01  1e-03  4e-14
 7: -8.4342e+00 -8.6789e+00  3e-01  3e-04  3e-14
 8: -8.4564e+00 -8.6002e+00  1e-01  9e-05  4e-14
 9: -8.4863e+00 -8.5480e+00  6e-02  2e-05  4e-14
10: -8.5059e+00 -8.5196e+00  1e-02  4e-06  4e-14
11: -8.5106e+00 -8.5131e+00  3e-03  5e-07  3e-14
12: -8.5113e+00 -8.5122e+00  9e-04  6e-08  4e-14
13: -8.5117e+00 -8.5117e+00  2e-05  1e-09  4e-14
14: -8.5117e+00 -8.5117e+00  2e-07  1e-11  4e-14
Optimal solution found.
115 support vectors out of 516 points
     pcost       dcost       gap    pres   d

     pcost       dcost       gap    pres   dres
 0: -7.4365e+01 -9.8324e+01  4e+03  3e+01  4e-13
 1: -1.0249e+01 -9.1708e+01  2e+02  9e-01  4e-13
 2: -7.6798e+00 -3.9794e+01  5e+01  2e-01  8e-14
 3: -7.1029e+00 -2.0191e+01  2e+01  5e-02  4e-14
 4: -7.1437e+00 -1.0711e+01  4e+00  1e-02  3e-14
 5: -7.4358e+00 -9.1365e+00  2e+00  4e-03  4e-14
 6: -7.5362e+00 -8.4527e+00  1e+00  9e-04  4e-14
 7: -7.7048e+00 -8.0793e+00  4e-01  2e-04  4e-14
 8: -7.7835e+00 -7.9209e+00  1e-01  5e-05  4e-14
 9: -7.8008e+00 -7.8891e+00  9e-02  2e-05  4e-14
10: -7.8262e+00 -7.8479e+00  2e-02  3e-06  4e-14
11: -7.8340e+00 -7.8371e+00  3e-03  4e-07  4e-14
12: -7.8352e+00 -7.8355e+00  2e-04  2e-08  4e-14
13: -7.8353e+00 -7.8353e+00  4e-06  4e-10  5e-14
Optimal solution found.
105 support vectors out of 516 points
     pcost       dcost       gap    pres   dres
 0: -4.2199e+01 -9.1567e+01  4e+03  3e+01  3e-13
 1: -6.2535e+00 -8.4591e+01  2e+02  1e+00  3e-13
 2: -4.2972e+00 -3.4148e+01  5e+01  2e-01  6e-14
 3: -3.59

13: -8.2247e+00 -8.2251e+00  4e-04  2e-09  4e-14
14: -8.2249e+00 -8.2249e+00  6e-06  3e-11  4e-14
Optimal solution found.
112 support vectors out of 516 points
     pcost       dcost       gap    pres   dres
 0: -3.8336e+01 -9.2552e+01  4e+03  3e+01  3e-13
 1: -5.6057e+00 -8.5499e+01  2e+02  9e-01  3e-13
 2: -3.8320e+00 -3.2488e+01  4e+01  2e-01  5e-14
 3: -2.9211e+00 -1.6038e+01  2e+01  7e-02  3e-14
 4: -2.3834e+00 -9.9775e+00  1e+01  4e-02  2e-14
 5: -1.9472e+00 -5.6463e+00  6e+00  2e-02  2e-14
 6: -1.6696e+00 -3.5295e+00  3e+00  6e-03  2e-14
 7: -1.6623e+00 -2.2054e+00  7e-01  1e-03  2e-14
 8: -1.6379e+00 -1.9648e+00  4e-01  6e-04  1e-14
 9: -1.6771e+00 -1.8051e+00  2e-01  2e-04  1e-14
10: -1.6973e+00 -1.7285e+00  4e-02  4e-05  1e-14
11: -1.7037e+00 -1.7085e+00  5e-03  4e-06  2e-14
12: -1.7052e+00 -1.7053e+00  6e-05  5e-08  2e-14
13: -1.7053e+00 -1.7053e+00  6e-07  5e-10  2e-14
Optimal solution found.
47 support vectors out of 516 points
     pcost       dcost       gap    pres   dr

13: -7.5682e+00 -7.5682e+00  2e-05  7e-10  4e-14
14: -7.5682e+00 -7.5682e+00  2e-07  8e-12  4e-14
Optimal solution found.
107 support vectors out of 516 points
     pcost       dcost       gap    pres   dres
 0: -4.4347e+01 -9.0371e+01  4e+03  3e+01  3e-13
 1: -6.5588e+00 -8.3428e+01  2e+02  9e-01  3e-13
 2: -4.6060e+00 -3.4688e+01  5e+01  2e-01  7e-14
 3: -3.6652e+00 -1.5939e+01  2e+01  7e-02  4e-14
 4: -3.0430e+00 -7.6007e+00  7e+00  2e-02  3e-14
 5: -2.7657e+00 -5.0635e+00  4e+00  1e-02  2e-14
 6: -2.5135e+00 -3.9318e+00  2e+00  5e-03  2e-14
 7: -2.4741e+00 -3.0985e+00  9e-01  2e-03  2e-14
 8: -2.4635e+00 -2.6618e+00  2e-01  2e-04  2e-14
 9: -2.5013e+00 -2.5711e+00  8e-02  7e-05  2e-14
10: -2.5108e+00 -2.5380e+00  3e-02  5e-16  2e-14
11: -2.5228e+00 -2.5239e+00  1e-03  2e-16  2e-14
12: -2.5233e+00 -2.5233e+00  1e-05  2e-16  2e-14
13: -2.5233e+00 -2.5233e+00  1e-07  2e-16  2e-14
Optimal solution found.
52 support vectors out of 516 points
     pcost       dcost       gap    pres   dr

In [12]:
# show the result
count = 0
for i in range(0,6):
    print("k=",name[0],"c=",c1[i],accuracy1[count],result1[count])
    count = count +1

k= linear c= 0.005 0.587692307692 27.0
k= linear c= 0.01 0.6 27.0
k= linear c= 0.03 0.575384615385 27.0
k= linear c= 0.05 0.561538461538 27.0
k= linear c= 0.07 0.566153846154 28.0
k= linear c= 0.09 0.575384615385 28.0
